This notebook tries out the state-of-the-art word embeding model [ELMo](https://allennlp.org/elmo).

In [13]:
import pandas as pd

from config import validate_data_path, train_data_path, testa_data_path
from fgclassifier import read_csv

def content_to_corpus(data_path, txt_path, sample_n=None):
    """Convert Review content to text corpus for word embedding training"""
    df = read_csv(data_path, seg_words=True, sample_n=None)
    if sample_n:
        df = df.sample(sample_n, random_state=1)
    all_content = '\n'.join(
        y.strip() for x in df['content']
        for y in x.strip('"').split('。') if y.strip()
    )
    print(all_content[:200])
    print()
    with open(txt_path, 'w') as f:
        f.write(all_content + '\n')
    
content_to_corpus(train_data_path, 'data/text_train.txt')
content_to_corpus(train_data_path, 'data/text_train_10k.txt', sample_n=10000)
content_to_corpus(validate_data_path, 'data/text_valid.txt')
content_to_corpus(testa_data_path, 'data/text_testa.txt')

2018-11-14 22:30:31,568 [INFO] Read cache data/train/sentiment_analysis_trainingset.csv.segged_sample_None.tsv..


吼吼 吼 ， 萌死 人 的 棒棒糖 ， 中 了 大众 点评 的 霸王餐 ， 太 可爱 了
一直 就 好奇 这个 棒棒糖 是 怎么 个 东西 ， 大众 点评 给 了 我 这个 土老冒 一个 见识 的 机会
看 介绍 棒棒糖 是 用 德国 糖 做 的 ， 不会 很甜 ， 中间 的 照片 是 糯米 的 ， 能 食用 ， 真是太 高端 大气 上档次 了 ， 还 可以 买 蝴蝶结 扎口 ， 送人 可以 买 



2018-11-14 22:30:36,007 [INFO] Read cache data/train/sentiment_analysis_trainingset.csv.segged_sample_None.tsv..
2018-11-14 22:30:39,507 [INFO] Read cache data/validate/sentiment_analysis_validationset.csv.segged_sample_None.tsv..


说起 这次 霸王餐 ， 真 有点 不知 如何 评价 ， 相当 的 一波三折
日期 从 十多号 改成 20 号 ， 然后 晚上 六点半 的 活动 ， 下午 两点 半 通知 改期 ， 再次 改到 23
到 了 23 号 终于 没 变动 了 ， 报名 时 的 全场 自助 默默 变成 了 套餐 ！ 除去 一些 不爱 吃 的 东西 ， 吃 到 的 东西 少得 可怜 ， 一个 人 三只 虾 也 是 醉 了 ！



2018-11-14 22:30:40,084 [INFO] Read cache data/test-a/sentiment_analysis_testa.csv.segged_sample_None.tsv..


哎 ， 想当年 来 佘山 的 时候 ， 啥 都 没有 ， 三品 香算 镇上 最大 看起来 最 像样 的 饭店 了
菜品 多 ， 有点 太 多 ， 感觉 啥 都 有 ， 杂都 不足以 形容
随便 点些 ， 居然 口味 什么 的 都 好 还 可以 ， 价钱 自然 是 便宜 当 震惊
元宝 虾 和 椒盐 九肚鱼 都 不错 吃
不过 近来 几次 么 ， 味道 明显 没 以前 好 了
冷餐 里面 一个 凉拌

我 想 说 他们 家 的 优惠活动 好 持久 啊 ， 我 预售 的 时候 买 的 券 ， 前两天 心血来潮 去 吃 的 活动 还 在 继续
首先 说 下 服务 ， 因为 和 男票 开车 去 的 ， 有点 不 认路 ， 老板 很 耐心 的 在 电话 里 帮 我们 指路 ， 到 了 门店 之后 也 帮 我们 推荐 了 他们 家 做 的 比较 地道 的 伤心 凉粉 ， 说 是 厨师 是 四川 那边 来 



In [14]:
!cat data/text_train.txt data/text_valid.txt data/text_testa.txt | wc

 1114130 30757624 166330158


All dataset combined has 30 million words, which is too much. According to [ELMoForManyLangs](https://github.com/HIT-SCIR/ELMoForManyLangs), it would take 3 days to train 20m words on an NVIDIA P100 GPU.

We must sample the data.

In [16]:
!wc data/text_train_10k.txt

   82727 2255688 12200210 data/text_train_10k.txt


In [17]:
!tail data/text_train_10k.txt

店里 冰淇淋 有 12 个 口味 的 ~
除了 冰淇淋 ~ 甜品 饮料 也 有 ~ 看 电影 前 可以 来 这里 等 ~ 也 可以 买 进去 吃 哈 ~
# 枫树 胡桃 # BBLANKK
店员 妹子 挖 完 冰淇淋 还 帮 我 称 了 一下 ~ 说 他们 家 的 单球 是 60g ~
味道 很 好 啊 ~ 奶味算重 的 了 ~ 也 不会 很甜 ~ 里面 有 核桃仁 ~ 核桃仁 也 好吃 ~
一个 球挺 多 的 ~ 吃 着 绝对 够 了 ~
不过 原价 有点 小贵 ~ M 团有 团购 能 便宜 一点 ~
总体 很 满意 的 ~ 以后 想 吃 冰淇淋 可以 来 这家 ~ 再 吃 吃 别的 味 ~
极食 urban BBLANKK harvest 是 一个 走 自然 环保 高端 线路 的 餐厅 ， 位于 杭州 大厦 D座 5 楼 ～ 光看 地标 也 是 够 逼格 了 一进 门口 装饰 有 微型 田园 BBLANKK 看上去 食材 都 新鲜 的 不要 不要 的 ～ 给 创意 一个 赞 BBLANKK 只要 菜单 上 有 “ 即 摘 ” 字眼 的 ， 都 由 服务员 现场 采摘 取用 ， 菜色 摆盘 精致 而且 色彩 丰富 ， 让 人 食欲 大开 ， 口味 都 还 不错 ～ 不过 本人 本 就 不是 一个 太 挑食 的 人 啦 饭后 上 了 甜品 ， 好吃 到 飞 起来 ， 吃 得 愉快 心情 就 好好 ， 尤其 是 这样 一个 周末
因为 价格 偏贵 的 缘故 ， 如此 黄金地段 、 高峰 时间 用餐 人 倒 不 多 ， 这 也 保证 了 用餐 环境 的 舒适度 ， 还有 一个 好消息 就是 ： 开业 初期 有 七折 优惠 哦 ！ 大家 赶紧 去 拔草 吧


A random sample of 10,000 reviews gives about 83k sentences, and 2.2m words. We expect the training
time for 10,000 reviews to be about 8 hours.

Let's try a very small subset just to make sure the software works.

In [22]:
# !shuf -n 10000 data/all_text.txt > data/little_text.txt
# Use `gshuf` on Mac
!gshuf -n 1000 data/text_train.txt > data/text_train_1k.txt
!gshuf -n 100 data/text_train.txt > data/text_train_100.txt
!wc data/text_train_1k.txt

    1000   29554  159470 data/text_train_1k.txt


In [24]:
# Train
# add `pwd` to `config_path` because trained model depends on that file
# absolute path makes it easier to find
!python -m elmoformanylangs.biLM train \
    --train_path data/text_train_1k.txt \
    --valid_path data/text_train_100.txt \
    --model data/elmo-zhs-1k \
    --config_path `pwd`/misc/elmo/cnn_50_100_512_4096_sample.json \
    --seed 1 \
    --gpu 0 \
    --optimizer adam \
    --lr 0.001 \
    --lr_decay 0.8 \
    --batch_size 32 \
    --clip_grad 5 \
    --max_epoch 10 \
    --max_sent_len 20 \
    --max_vocab_size 150000 \
    --eval_steps 10000 \
    --min_count 5

Namespace(batch_size=32, clip_grad=5.0, config_path='/Users/jesse/workspace/ML/fine-grain-sentiment-analysis/misc/elmo/cnn_50_100_512_4096_sample.json', eval_steps=10000, gpu=0, lr=0.001, lr_decay=0.8, max_epoch=10, max_sent_len=20, max_vocab_size=150000, min_count=5, model='data/elmo-zhs-1k', optimizer='adam', save_classify_layer=False, seed=1, test_path=None, train_path='data/text_train_1k.txt', valid_path='data/text_train_100.txt', valid_size=0, word_embedding=None)
{'encoder': {'name': 'elmo', 'projection_dim': 512, 'cell_clip': 3, 'proj_clip': 3, 'dim': 4096, 'n_layers': 2}, 'token_embedder': {'name': 'cnn', 'activation': 'relu', 'filters': [[1, 32], [2, 32], [3, 64], [4, 128], [5, 256], [6, 512], [7, 1024]], 'n_highway': 2, 'word_dim': 100, 'char_dim': 50, 'max_characters_per_token': 50}, 'classifier': {'name': 'sampled_softmax', 'n_samples': 8192}, 'dropout': 0.1}
2018-11-14 22:36:57,640 INFO: training instance: 1505, training tokens: 28596.
2018-11-14 22:36:57,645 INFO: valid i

In [25]:
df_train = read_csv(train_data_path, seg_words=True, sample_n=None)

2018-11-14 23:19:31,908 [INFO] Read cache data/train/sentiment_analysis_trainingset.csv.segged_sample_None.tsv..


In [28]:
from elmoformanylangs import Embedder

elmoemb = Embedder('data/elmo-zhs-fsauor')
# e = Embedder('data/elmo-zhs-1k')

sents = df_train['content'][0:2]
elmoemb.sents2elmo(sents)

2018-11-14 23:21:26,090 [INFO] char embedding size: 8582
2018-11-14 23:21:26,861 [INFO] word embedding size: 69827
2018-11-14 23:21:34,862 [INFO] Model(
  (token_embedder): ConvTokenEmbedder(
    (word_emb_layer): EmbeddingLayer(
      (embedding): Embedding(69827, 100, padding_idx=3)
    )
    (char_emb_layer): EmbeddingLayer(
      (embedding): Embedding(8582, 50, padding_idx=8579)
    )
    (convolutions): ModuleList(
      (0): Conv1d(50, 32, kernel_size=(1,), stride=(1,))
      (1): Conv1d(50, 32, kernel_size=(2,), stride=(1,))
      (2): Conv1d(50, 64, kernel_size=(3,), stride=(1,))
      (3): Conv1d(50, 128, kernel_size=(4,), stride=(1,))
      (4): Conv1d(50, 256, kernel_size=(5,), stride=(1,))
      (5): Conv1d(50, 512, kernel_size=(6,), stride=(1,))
      (6): Conv1d(50, 1024, kernel_size=(7,), stride=(1,))
    )
    (highways): Highway(
      (_layers): ModuleList(
        (0): Linear(in_features=2048, out_features=4096, bias=True)
        (1): Linear(in_features=2048, out_f

[array([[ 0.16197295,  0.05117923, -0.13360588, ...,  0.24924016,
         -0.27383432, -0.05000202],
        [ 0.06199374,  0.07696173, -0.23577517, ...,  0.26290917,
         -0.27809614, -0.06464732],
        [ 0.0008731 , -0.04712557, -0.08987609, ..., -0.00968068,
         -0.13784361, -0.03070146],
        ...,
        [-0.27461815,  0.03719755, -0.13428287, ..., -0.06027537,
         -0.07961495, -0.14139868],
        [-0.06108154,  0.13908525, -0.2571093 , ...,  0.01645198,
         -0.02265188, -0.02150639],
        [-0.19907278,  0.02419554, -0.19596738, ...,  0.07171986,
         -0.01507132,  0.02613006]], dtype=float32),
 array([[-0.17354196, -0.36698878, -0.04885902, ..., -0.02680009,
          0.13608061,  0.12091058],
        [-0.29550084, -0.12204532, -0.28084293, ..., -0.29321548,
          0.13516046, -0.11846065],
        [-0.05146622, -0.19961458,  0.01509347, ...,  0.05483557,
         -0.06374971, -0.0203303 ],
        ...,
        [-0.1181826 ,  0.02767085, -0.3

Now let's train the full model.

In [ ]:
!cat data/text_train.txt data/text_testa.txt > data/text_train_valid.txt

In [ ]:
# To train the full model
!time python -m elmoformanylangs.biLM train \
    --train_path data/text_train_valid.txt \
    --valid_path data/text_testa.txt \
    --model data/elmo-zhs \
    --config_path `pwd`/misc/elmo/cnn_50_100_512_4096_sample.json \
    --seed 1 \
    --gpu 0 \
    --optimizer adam \
    --lr 0.001 \
    --lr_decay 0.8 \
    --batch_size 32 \
    --clip_grad 5 \
    --max_epoch 10 \
    --max_sent_len 20 \
    --max_vocab_size 150000 \
    --eval_steps 10000 \
    --min_count 5